# RTS Data Processing


In [16]:
import json
import tokenize
import numpy as np
import pandas as pd
import os
import re
from itertools import islice


BASE_PATH ='data/RTS_dataset_per_year/{}/{}'
parties = r"\bUDC\b|\bVerts\b|\bPS\b|\bPDC\b|\bPLR\b|\bPES\b|Vert lib|Vert'lib|Verte lib|Vert-lib|Verts lib|Verts-lib|Verte-lib|\bVerte\b|\bVert\b|\bPVL\b|\bPSS\b|\bSocialiste\b"
thematicGeographicals = r'Geneve|Suisse|Valais|Berne|Zurich|Vaud|Fribourg|Neuchatel|Sion|canton|Bale|Vevey|Jura|Montreux|Bienne|GRUYÈRE|Lucerne|TESSIN|Winterthour|Lugano'
thematicThemes = r"DÉBAT|POLITIQUE|LÉGISLATIF|élection|exécutif|conseiller|initiative populaire|votation|référendum|PRÉSIDENT|Vote|CANDIDAT|CAMPAGNE"
YEARS = np.arange(2010, 2020)

In [23]:
#Next, we want to go year by year and find interviews that are with interviews of political parties
party_dict =[]
contributor_dict = []


def read_in(party):   
    
    for year in YEARS:
        if os.path.exists(BASE_PATH.format(year, party)):
            files = [f for f in os.listdir(BASE_PATH.format(year, party))]
            for file in files:
                with open(BASE_PATH.format(year, party)+"/{}".format(file), 'r', encoding='utf-8') as fp:
                    dicts = json.load(fp)
                    for entry in dicts['data']:
                        
                        #Here we will build a dictionary for looking at the contributors of RTS
                        if 'contributors' in entry and 'publicationDate' in entry:
                            contributor_dict.append(entry)
                        
                        #Here we are filtering through the json to build documents for the topic extraction
                        if ('speechToTexts' or 'subTitles' or 'summary') in entry and ('publicationDate' in entry):
                            if 'genres' in entry:
                                if 'Interview' in entry['genres']:
                                    if 'thematicGeographicals' in entry:
                                        for place in entry['thematicGeographicals']:
                                            if re.search(thematicGeographicals,place, re.I):
                                                party_dict.append(entry)
                                    else:
                                        party_dict.append(entry)
                            if 'thematicThemes' in entry and 'thematicGeographicals' in entry:
                                for place in entry['thematicGeographicals']:
                                    if re.search(thematicGeographicals, place, re.I):
                                        for theme in entry['thematicThemes']:
                                            if re.search(thematicThemes, theme, re.I):
                                                party_dict.append(entry)
                            if 'title' in entry:
                                if re.search(parties, entry['title'], re.I):
                                    party_dict.append(entry)
                            if 'thematicPersons' in entry: 
                                for person in entry['thematicPersons']:
                                    if re.search(parties, person, re.I):
                                        party_dict.append(entry)
                            if 'chapters' in entry:
                                for chapter in entry['chapters']:
                                    if 'excerpt' in chapter:
                                        if re.search(parties, chapter['excerpt'], re.I):
                                            party_dict.append(chapter)
                                        if 'thematicThemes' in chapter and 'thematicGeographicals' in chapter:
                                            for place in chapter['thematicGeographicals']:
                                                type(place)
                                                if re.search(thematicGeographicals, place, re.I):
                                                    for theme in chapter['thematicThemes']:
                                                        if re.search(thematicThemes, theme, re.I):
                                                            party_dict.append(chapter)

    return 

#For each party, convert the json into dictionaries

read_in('UDC')
read_in('PDC')
read_in('PS')
read_in('PLR')
read_in('PES')
read_in('PVL')
read_in('Verts')
read_in('Politque')

In [24]:
all_party_topic = pd.DataFrame.from_dict(party_dict)
all_party_contributor = pd.DataFrame.from_dict(contributor_dict)

Above, the json created by querying the RTS archive API for each of the party abbreviations was created into lists of dictionaries. However, it was noted that RTS does not often use the abbreviations for the PES and PVL parties and prefers to use the Verts and Verts libéreaux. Another query was made using the key word "verts" to find more broadcasts for these parties. Below these broadcasts were sorted into either verts or verts libéraux

After reading in the lists of dictionaries, each dictionary is converted to a pandas dataframe. During on our analysis we will look at either the "speechToTexts" or "summary" collumns in order to do some NLP to determine what the key issues each party were discussing and potentially their sentiment about these topics.

This next part is for processing the SpeechtoText portions of the interviews, first we analyse what percentage of our desired groups in fact have the SpeechtoText feature

Below we are cleaning up the data to get ready to make the topic model. A new attribute of year is added, subTitles and speechToTexts are combined with a preference to subtitles and then all duplicate entries are eliminated

In [25]:
#Converting all possible sources of text to one column
year = all_party_topic["publicationDate"].str[:4]
all_party_topic['year'] = year
all_party_topic['text'] = all_party_topic.apply(lambda x: x['subTitles'] if x['speechToTexts'] == None else x['speechToTexts'], axis=1)
all_party_topic['text'] = all_party_topic.apply(lambda x: x['excerpt'] if x['text'] == None else x['text'], axis=1)
all_party_topic['text'] = all_party_topic.apply(lambda x: x['summary'] if x['text'] == None else x['text'], axis=1)

In [26]:
year = all_party_contributor["publicationDate"].str[:4]
all_party_contributor['year'] = year

In [27]:
#Drop any duplicates
print(len(all_party_contributor))
all_party_contributor= all_party_contributor.drop_duplicates('internalArchiveId')
print(len(all_party_contributor))

print(len(all_party_topic))
all_party_topic= all_party_topic.drop_duplicates('internalArchiveId')
print(len(all_party_topic))

31306
17995
26011
7131


Below we will be looking at the number of times a representative from each party is a contributor on RTS.

In [7]:
all_party_contributor = all_party_contributor[['contributors', 'publicationDate', 'year', 'internalArchiveId']]


In [8]:
all_party_contributor = all_party_contributor[['contributors', 'publicationDate', 'year', 'internalArchiveId']]
contributors = all_party_contributor.explode('contributors')
contributors['contributor_role'] = contributors.apply(lambda x: str(x['contributors']['role']) if 'role' in x['contributors'] else x['contributors'] == None, axis=1)


In [10]:
for contributor in contributors['contributor_role'].unique():
    print(contributor)

False
maître d'enseignement et de recherche en sciences politiques à l'Université de Lausanne
conseiller national valaisan UDC
collaborateur/trice RSR
correspondant RSR Paris
conseillère nationale genevoise socialiste
conseiller national jurassien UDC
correspondante RSR Palais fédéral
conseillère fédérale: cheffe du DFJP
conseiller national genevois UDC
correspondant RSR Palais fédéral
conseiller national valaisan PDC
président de l'Assemblée fédérale
conseiller national fribourgeois UDC
conseiller aux Etats neuchâtelois socialiste
collaborateur RSR
ancien conseiller fédéral, vice-président de l'UDC suisse
linguiste à l'Université de Lausanne et spécialiste de communication politique
conseiller en communication, directeur Farner Consulting
auditeur/trices RSR
vice-président du groupe UDC aux chambres fédérales, conseiller national fribourgeois
Politologue, spécialiste des nationalismes et du populisme 
politologue, spécialiste des mouvements d'extrême droite en Europe, chercheur
consei

CN UDC VD
Conseiller national socialiste neuchâtelois / président d'Helvetia Latina 
envoyé spécial RTS à Londres
ancien ambassadeur de Suisse en Grande-Bretagne
analyste au Think Tank Open Europe
correspondante RTS à Londres
député européen écologiste français
Duplex Peterborough
Duplex St-Etienne
Liaison duplex Londres
Liaison duplex Peterborough
Ex Conseiller fédéral
correspondant RTS Jura bernois
conseiller d'État UDC bernois
Invité Plateau
Duplex Montpellier
directeur du département de politique générale au Centre patronal Vaudois
CN, vice-présidente UDC
Ex-CN(/CE UDC NE 
liaison duplex Yverdon
Liaison duplex Delémont
conseiller national PS fribourgeois
sociologue des religions
Co-président 
Conseiller national UDC/ Schaffhouse 2007-
Liaison duplex Washington
Pdt (élu avril 2016) Ancien Conseiller aux Etats  - La Poste 
Duplex Aarau
Pdte, députée, conseillère nationales 
Conseiller administratif, Dpt de l'environnement urbain et de la sécurité, CN  
Duplex de Berne
Présidente / Ca

président du conseil d'administration de la Poste
vice-président de la société suisse des médecins de famille
surveillant des prix et membre du PDC
conseiller aux États PLR vaudois
conseillère aux États socialiste vaudoise et vice-présidente du PSS
conseiller national PDC zougois et président du PDC
conseiller d'État PLR vaudois et chef du département de l'économie
conseiller d'État PDC genevois, chef du Département des transports et de l'environnement
Duplex Paris Roland Garros
politologue, directeur du centre d'études européennes de l'Université de Fribourg
conseiller aux Etats PLR argovien
député et président du PDC genevois
Professeur, CN PLR AR 
conseillère aux Etats jurassienne, membre de la présidence du PDC suisse
artiste franco-suisse, 
membre du bureau des Femmes PDC suisse 
Duplex Aigle
Président, association suisse des producteurs de sucre
Chef des Ressources humaines - Migros
Montfaucon
Conseillère d'Etat PS GE en chargée du département de l'instruction publique, de la cul

Directeur de l'interprofession du Vacherin Fribourgeois et député PLR au Grand Conseil Fribourgeois., 
Belge
Producteur, Dicodeur
rédactrice en chef de "Bon à Savoir", magazine des consommateurs suisses romands
rédacteur en chef adjoint RTSinfo.&#x0D;
rédacteur en chef adjoint RTSinfo&#x0D;

rédacteur en chef du quotidien "L'Agefi"
président du comité d'initiative pour 2 hôpitaux
Faux duplex Pyeonchang
conseiller national vaudois PLR, rédacteur en chef de l'AGEFI
président romand du comité "No Billag"
chef de la clinique de chirurgie viscérale et directeur du département de chirurgie des HUG
conseiller fédéral : chef du Département fédéral des Affaires étrangères (DFAE)
conseiller fédéral, chef du Département fédéral de l'économie, de la formation et de la recherche (DEFR)
associé de la banque Lombard Odier
président du groupe Kudelski
membre de Greenpeace suisse
militants opposés au projet d'aéroport à Notre-Dame-des-Landes
Duplex Swiss Expo Lausanne
conseiller national PLR neuchâtelo

In [11]:
#For each party, look at the contributor and determine if they were from the party or not
contributors['UDC'] = contributors.apply(lambda x: x['contributor_role'] if re.search(r'\bUDC\b', str(x['contributor_role'])) else None, axis =1)
contributors['PDC'] = contributors.apply(lambda x: x['contributor_role'] if re.search(r'\bPDC\b', str(x['contributor_role'])) else None, axis =1)
contributors['PS'] = contributors.apply(lambda x: x['contributor_role'] if re.search(r'\bPS\b|\bPSS\b|\bsocialise\b|\bsocialiste\b', str(x['contributor_role'])) else None, axis =1)
contributors['PLR'] = contributors.apply(lambda x: x['contributor_role'] if re.search(r'\bPLR\b', str(x['contributor_role'])) else None, axis =1)
contributors['Verts'] = contributors.apply(lambda x: x['contributor_role'] if re.search(r"Verts|Vert|\bPES\b|\bPVL\b", str(x['contributor_role']), re.I) else None, axis =1)
contributors['Verts_Libereux'] = contributors.apply(lambda x: x['Verts'] if re.search(r"Vert lib|Vert'lib|Verte lib|Vert-lib|Verts lib|Verts-lib|Verte-lib|\bPVL\b", str(x['Verts']), re.I) else None, axis =1)
contributors['Verts'] = contributors.apply(lambda x: x['Verts'] if x['Verts_Libereux'] == None else None, axis =1)


In [14]:
#Create a pie chart of the total political contributors and their respective parties
contributors[['UDC', 'PDC', 'PS', 'PLR', 'Verts', 'Verts_Libereux', 'year']].groupby('year').count()

,UDC,PDC,PS,PLR,Verts,Verts_Libereux
year,,,,,,
2010,47,28,38,14,14,0
2011,93,35,74,21,29,10
2012,82,63,65,48,21,3
2013,74,66,60,71,26,4
2014,97,41,60,52,28,3
2015,141,89,123,116,50,20
2016,102,58,98,103,42,9
2017,104,58,140,172,72,14
2018,129,100,178,170,87,7


In [15]:
contributors[['UDC', 'PDC', 'PS', 'PLR', 'Verts', 'Verts_Libereux']].count()

UDC               977
PDC               618
PS                989
PLR               938
Verts             490
Verts_Libereux     92
dtype: int64

We can see from above that the potentially useful columns to analyze the broadcasts would be the summary and speechToTexts. At first we will focus on the speechToTexts

Now we want to look at the words used in the interviews, by both the interviewer and the interviewee using Robin's bag of words method


In [28]:
import re

str_YEARS = ['2019', '2018', '2017', '2016', '2015', '2014', '2013', '2012','2011','2010']


def speech_to_words(speech, year=str_YEARS):
    """
    Return a bag of word sorted by frequency
    
    lang: {'fr','de'}
    username: {'PS Suisse', 'CVP PDC PPD PCD', 'UDC Suisse',
       'Les VERTS suisses 🌻', "Vert'libéraux Suisse", 'PLR Suisse'}
    year: default = [2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012, 2011, 2010]
    
    """
    speech_copy = speech.copy()
    speech_copy = speech_copy[speech_copy['year'].isin(year)]
    speech_copy = speech_copy["text"].astype(str).str.lower()
    
    for text in speech_copy:
        yield(gensim.utils.simple_preprocess(str(text), deacc=True))  # deacc=True removes punctuations

In [29]:
import nltk
from nltk.corpus import stopwords
stopwords_fr = stopwords.words('french')
stopwords_fr.extend(['nlocuteur'])

Investigation for using Spacy

In [30]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
# spacy for lemmatization
import spacy

from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [31]:
data_words = list(speech_to_words(all_party_topic))


Creating bigrams

In [32]:
# Build the bigram model
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=75) # higher threshold fewer phrases.


In [33]:
# Faster way to get a sentence clubbed as a bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)


Remove stop words, make birgams and lemmatize

In [ ]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords_fr] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'fr_core_news_md' model
nlp = spacy.load('fr_core_news_md', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

Creation of the topic model

In [20]:
#Saving the politique only model
.save('models/RTS_models/all_politique.model')

In [18]:
import os 
from gensim.models.wrappers import LdaMallet
os.environ.update({'MALLET_HOME': r'C:/Users/Isabelle/Downloads/mallet-2.0.8/mallet-2.0.8/'})
mallet_path = 'C:\\Users\\Isabelle\\Downloads\\mallet-2.0.8\\mallet-2.0.8\\bin\\mallet'

ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word) # update this path


# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(12,
  [('oui', 0.06346635381463517),
   ('alors', 0.029058467141781504),
   ('bien', 0.025613300880771857),
   ('faire', 0.024159939764310353),
   ('bon', 0.023617118142499432),
   ('dire', 0.022470188586737643),
   ('aller', 0.02211560349507083),
   ('ca', 0.014192158854121067),
   ('meme', 0.011806370274387575),
   ('tout', 0.011412386839202227)]),
 (6,
  [('faire', 0.022891337386018236),
   ('dire', 0.022511398176291792),
   ('donc', 0.017564881926584053),
   ('meme', 0.014328092120645312),
   ('question', 0.01402121814355857),
   ('droit', 0.012589139583820436),
   ('bien', 0.011748889408463876),
   ('aussi', 0.009922258592471358),
   ('tout', 0.009418108487257423),
   ('autre', 0.008497486555997195)]),
 (7,
  [('initiative', 0.041403988250790676),
   ('suisse', 0.03159889270041629),
   ('loi', 0.02525234737650299),
   ('peuple', 0.017384321708566075),
   ('vote', 0.014883740587601344),
   ('dire', 0.013862376467770679),
   ('question', 0.012207062204596842),
   ('oui', 0.0111575

Now we want to find the optimal number of topics by looking at their coherence values

In [ ]:
import os 
from gensim.models.wrappers import LdaMallet
os.environ.update({'MALLET_HOME': r'C:/Users/Isabelle/Downloads/mallet-2.0.8/mallet-2.0.8/'})
mallet_path = 'C:\\Users\\Isabelle\\Downloads\\mallet-2.0.8\\mallet-2.0.8\\bin\\mallet'

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=2, limit=60, step=6)
# Show graph
limit=60; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()



In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [155]:
optimal_model = model_list[6]
model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=15))

[(23,
  '0.024*"faire" + 0.018*"marche" + 0.018*"aussi" + 0.017*"prix" + '
  '0.016*"donc" + 0.015*"suisse" + 0.013*"consommateur" + 0.013*"dire" + '
  '0.012*"alors" + 0.009*"bien" + 0.009*"aller" + 0.009*"sant" + '
  '0.008*"falloir" + 0.008*"produire" + 0.007*"voir"'),
 (24,
  '0.053*"canton" + 0.025*"pourcent" + 0.023*"oui" + 0.021*"dire" + '
  '0.019*"eter" + 0.018*"aussi" + 0.017*"tre" + 0.016*"resultat" + '
  '0.014*"vote" + 0.013*"aller" + 0.013*"meme" + 0.012*"donc" + '
  '0.011*"campagne" + 0.010*"bien" + 0.009*"suisse"'),
 (15,
  '0.021*"faire" + 0.021*"dire" + 0.015*"question" + 0.015*"autre" + '
  '0.014*"aussi" + 0.013*"ecole" + 0.012*"droit" + 0.011*"meme" + '
  '0.011*"falloir" + 0.010*"valeur" + 0.010*"donc" + 0.010*"bien" + '
  '0.010*"tout" + 0.010*"religion" + 0.009*"cas"'),
 (3,
  '0.077*"initiative" + 0.029*"peuple" + 0.027*"suisse" + 0.020*"loi" + '
  '0.020*"initiativ" + 0.020*"vote" + 0.015*"texte" + 0.014*"projet" + '
  '0.013*"votation" + 0.013*"constitution"

In [179]:
#Saving optimal LDA Mallet model
optimal_model.save('models/RTS_models/LDAMallet_40topics.model')